In [1]:
import numpy as np
import pandas as pd
import random

In [2]:
from sklearn.model_selection import train_test_split

In [3]:
df = pd.read_csv("/Users/florian/Documents/GitHub/4 Semester/APPD/Analyse_von_Prozessdaten/setup/SmA-Four-Tank-Batch-Process_V2.csv", sep= ";")

In [4]:
import numpy as np

class SOM:
    def __init__(self, input_dim, output_dim, lr, max_iter):
        self.input_dim = input_dim
        self.output_dim = output_dim
        self.lr = lr
        self.max_iter = max_iter
        self.weights = np.random.rand(output_dim[0], output_dim[1], input_dim)
    
    def distance(self, x, y):
        # Euclidean distance between two vectors
        return np.sqrt(np.sum(np.power(x - y, 2)))
    
    def neighbourhood(self, center, radius):
        # Gaussian neighbourhood function
        d = np.zeros(self.output_dim)
        for i in range(self.output_dim[0]):
            for j in range(self.output_dim[1]):
                d[i,j] = self.distance(center, np.array([i,j]))
        return np.exp(-np.power(d, 2) / (2 * np.power(radius, 2)))
    
    def fit(self, X):
        for iter_count in range(self.max_iter):
            # Select a random input sample
            x = X[np.random.randint(X.shape[0])]
            
            # Find the best matching unit (BMU)
            bmu_idx = np.array([0, 0])
            min_dist = np.iinfo(np.int).max
            for i in range(self.output_dim[0]):
                for j in range(self.output_dim[1]):
                    w = self.weights[i,j,:]
                    d = self.distance(x, w)
                    if d < min_dist:
                        min_dist = d
                        bmu_idx = np.array([i, j])
            
            # Update BMU and neighbourhood weights
            radius = min(self.output_dim[0], self.output_dim[1]) / 2
            neighbourhood = self.neighbourhood(bmu_idx, radius)
            for i in range(self.output_dim[0]):
                for j in range(self.output_dim[1]):
                    w = self.weights[i,j,:]
                    d = np.array([i,j]) - bmu_idx
                    influence = neighbourhood[i,j] * self.lr
                    new_w = w + influence * (x - w)
                    self.weights[i,j,:] = new_w
                    
        print("Training complete.")
    
    def detect_anomalies(self, X, threshold=0.5):
        # Calculate node frequencies
        node_freq = np.array(self.node_freq)
        node_freq_norm = node_freq / np.sum(node_freq)
        
        # Calculate average distances
        dist_avg = np.zeros(self.output_dim)
        for i in range(self.output_dim[0]):
            for j in range(self.output_dim[1]):
                w = self.weights[i,j,:]
                X_mapped = X[self.node_map == (i,j),:]
                if X_mapped.shape[0] > 0:
                    dist_avg[i,j] = np.mean([self.distance(x, w) for x in X_mapped])
                else:
                    dist_avg[i,j] = np.nan
        
        # Identify anomalous nodes
        node_anomalous = np.logical_or(node_freq_norm < threshold, 
                                       dist_avg > np.nanmean(dist_avg) + np.nanstd(dist_avg))
        
        # Identify anomalous data points
        X_anomalous = []
        for i in range(X.shape[0]):
            x = X[i,:]
            bmu_idx = np.array([0, 0])
            min_dist = np.iinfo(np.int).max
            for i in range(self.output_dim[0]):
                for j in range(self.output_dim[1]):
                    w = self.weights[i,j,:]
                    d = self.distance(x, w)
                    if d < min_dist:
                        min_dist = d
                        bmu_idx = np.array([i, j])
            if node_anomalous[bmu_idx[0], bmu_idx[1]]:
                X_anomalous.append(x)
        
        return node_anomalous, np.array(X_anomalous)

In [5]:
df1 = df.drop(["DeviationID ValueY", "timestamp"], axis= 1)

In [6]:
randomlist = random.sample(range(0, 260), 78)

X_train = df1.loc[df.index.isin(randomlist)]
X_test = df1.loc[~df.index.isin(randomlist)]


In [7]:
som = SOM(input_dim= 42, output_dim= (42,42), lr= 0.1, max_iter=1000)

In [12]:
som.fit(X_train)

KeyError: 25